In [1]:
import pandas as pd
import numpy as np
from scipy.stats.contingency import chi2_contingency
from scipy.stats import chi2
from tqdm import tqdm

# Funções e dicionários

In [ ]:
z_dict = {
    .90: 1.645,
    .95: 1.96
}

# z_dict = {99:   2.575,
#           95:   1.960,
#           90:   1.645,
#           70:   1.035,
#           67.5: 0.985}

In [ ]:
def classificacao_nps(x):
    if x <= 6:
        return 'detrator'
    if x <= 8:
        return 'neutro'
    if x > 8:
        return 'promotor'
    else:
        return None

In [ ]:
def tabela_esperado(df_, colunas, filtros = None):
    
    if filtros is not None:
        
        df = df_.copy()
    
        for k, v in filtros.items():
            df = df[df[k].isin(v)]
            
    else:
        df = df_.copy()

    chi_sqr = df[colunas + ['NPS_GERAL']].copy()
    chi_sqr['class_nps'] = chi_sqr['NPS_GERAL'].apply(classificacao_nps)
    chi_sqr = chi_sqr[colunas + ['class_nps']].value_counts().reset_index(name = 'qtd')

    return chi_sqr

def relatorio_qui_quadrado(col1, col2, filtros = None, print_relatorio = True):

    base_esperado = tabela_esperado(colunas = [col1, col2], filtros = filtros)
    
    tabela_cat_1 = pd.DataFrame()

    for cat1 in base_esperado[col1].unique():

        if print_relatorio:
            print(cat1.upper())
            print('-'* 100)

        tabela_contingencia = []

        for cat2 in base_esperado[col2].unique():
            lista = list(base_esperado[(base_esperado[col1] == cat1) & (base_esperado[col2] == cat2)]['qtd'])

            tabela_contingencia.append(lista)

        tabela_contingencia = np.array(tabela_contingencia)

        # Aplicando o teste qui-quadrado
        alpha = 0.05
        chi2_stat, p, dof, expected = chi2_contingency(tabela_contingencia)
        valor_critico = chi2.ppf(1 - alpha, dof)
        
        if print_relatorio:

            # Exibindo os resultados
            print(f"Qui-quadrado: {chi2_stat:.2f}")
            print(f"Valor crítico: {valor_critico:.3f}")
            print(f"P-valor: {p:.4f}")
            print(f"Graus de liberdade: {dof}")
            print()
            print("Frequências observadas:")
            print(tabela_contingencia)
            print()
            print("Frequências esperadas:")
            print(expected.round())
            print()
            print("Observado - esperado:")
            print(tabela_contingencia - expected.round())
            print()

            # Interpretação
            print('Resultado:')
            if p < alpha:
                print("Rejeitamos a hipótese nula: há uma associação significativa entre os grupos e as categorias.")
            else:
                print("Não rejeitamos a hipótese nula: não há evidências de associação significativa entre os grupos e as categorias.")

            print()
        
        tmp = {col1:[cat1],
               'qui_quadrado':[f"{chi2_stat:.2f}"],
               'valor_critico':[f"{valor_critico:.3f}"],
               'p_valor':[f"{p:.4f}"]}
        
        df_tmp = pd.DataFrame(data = tmp)
        
        tabela_cat_1 = pd.concat([tabela_cat_1, df_tmp])
        
    return tabela_cat_1

def margem_erro(p, n, d, z):

    total = p + n + d

    nps = (p - d) / total 

    var_nps = ((1 - nps)**2 * (p/total)) + ((0 - nps)**2 * (n/total)) + ((1 - nps)**2 * (d/total))

    margem_erro = round(z * (var_nps/total)**(1/2) * 100, 1)
    
    return margem_erro

def margem_erro_grupo(df, z):
    
    classificacoes = ['promotor', 'neutro', 'detrator']
    valores = {classe: 0 for classe in classificacoes}
    
    for classe in classificacoes:
        if classe in df.class_nps.unique():
            valores[classe] = df[df['class_nps'] == classe].reset_index()['qtd'][0]
    
    margem = margem_erro(p = valores['promotor'], n = valores['neutro'], d = valores['detrator'], z = z)
    
    return margem


# Gerar tabela de NPS usando bootstrap com repetição

In [ ]:
# Bootstrap com repetição
df_margem_erro = pd.DataFrame()

for plataforma in ['Controle', 'Pós-pago']:
    for cat in ['E-mail', 'WhatsApp', 'Correios', 'Aplicativo']:
        for multa in ['Teve multa', 'Não teve multa']:
            
            print(f"{plataforma} | {cat} | {multa}")
            
            df_tmp = base_analitica_nps[(base_analitica_nps['COD_PLAT'] == plataforma) & 
                                        (base_analitica_nps['FLG_MULTA_ULTIMOS_12_MESES'] == multa) &
                                        (base_analitica_nps['DSC_CATEGORIA_IMPRESSAO'] == cat)]\
                .copy().reset_index(drop = True)


            for i in tqdm(range(10000)):

                amostra_i = np.random.randint(low = 0, high = df_tmp.shape[0], size = df_tmp.shape[0]).tolist()
                df_i = df_tmp.loc[amostra_i].copy()
                df_i['class_nps'] = df_i['NPS_GERAL'].apply(classificacao_nps)

                df_nps = df_i['class_nps'].value_counts()

                total = df_nps.sum()
                p = df_nps['promotor'] * 100 / total
                d = df_nps['detrator'] * 100 / total

                nps = round(p - d)

                tmp = pd.DataFrame(data = {'COD_PLAT':[plataforma],
                                           'DSC_CATEGORIA_IMPRESSAO':[cat],
                                           'FLG_MULTA_ULTIMOS_12_MESES':[multa],
                                           'nps': [nps]})

                df_margem_erro = pd.concat([df_margem_erro, tmp])